In [ ]:
import os
import pygplates
import numpy as np
import pyvista as pv
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from scipy.interpolate import griddata
from scipy.spatial.transform import Rotation as R

props = {}

To avoid having to pass too many variables later on in this project, all of our simulation properties will be specified in a dictionary named *props*. Later we will build a function that takes *props* as its only input and runs the entire simulation from it.

In [ ]:
props['earthRadius'] = 6371 #Radius of earth
props['heightAmplificationFactor'] = 60 #How much to amplify topography in visualizations

#Properties representing time range and time steps of our simulation
props['startTime'] = 10
props['endTime'] = 0
props['deltaTime'] = 5

#Properties specifying the location of data files
props['platePolygonsDirectory'] = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz'
props['rotationsDirectory'] = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot'

# Read Data and Spherical Polar Coordinate Transformations

To initiate our earth, we read a data files containing the Longatude, Latitude, and Elevation of the earth at various times. These data files were downloaded from:
<br> https://www.earthbyte.org/paleodem-resource-scotese-and-wright-2018/

In [ ]:
#Read initial landscape data at specified time from file which is in the form of (lon, lat, height)
def getInitialEarth(time, paleoDemsPath='./PaleoDEMS'):
    
    #Get path of initial landscape data file at specified time
    paleoDemsPath = Path('./PaleoDEMS')
    initialLandscapePath = list(paleoDemsPath.glob('**/*%03.fMa.csv'%time))[0]
    
    #Read data and split by newline and commas to create numpy array of data
    initialLandscapeFileLines = open(initialLandscapePath).read().split('\n')[1:-1]
    initLandscapeData = [line.split(',') for line in initialLandscapeFileLines]
    initLandscapeData = np.array(initLandscapeData).astype(float)
    
    #Set heights from metres to kilometers and return data
    initLandscapeData[:, 2] /= 1000
    return initLandscapeData

#Create mesh of the initial data
initData = getInitialEarth(props['startTime'])
initialDataMesh = pv.PolyData(initData).delaunay_2d()

#Create a plotter for visualizing the initial data
initialPlotter = pv.Plotter(notebook=True)
initialPlotter.add_mesh(initialDataMesh, scalars=initData[:, 2])
initialPlotter.camera_position = 'xy'
initialPlotter.camera.position = (0, 0, 350.0)
initialPlotter.show(jupyter_backend='panel', window_size=[800, 400])

The earth here seems a bit flat, but apparently its meant to be a sphere. To transform a coordinates from cartesian (XYZ) to spherical polar coordinates $(r, \theta, \phi)$ we define a few coordinate transformation functions. Our coordinate transformations are given by


$r = \sqrt{x^2 + y^2 + z^2}$ <br>
$\theta = \tan^{-1} (\frac{y}{x})$ <br>
$\phi = \cos^{-1} (\frac{z}{r})$

and to convert back to cartesian coordinates:

$x = r \cos \theta \sin \phi$ <br> 
$y = r \sin \theta \sin \phi$ <br>
$z = r \cos \phi$


Normally these functions take angles in the form of radians, however by setting *useLonLat* to True, the functions can take longitude and latitude as coordinates, which is a more common format in geosciences. For more details about spherical polar coordinates, see the article linked bellow: <br>
https://mathworld.wolfram.com/SphericalCoordinates.html

<div>
<img src="files/Images/SphericalTransforms.jpg" width="600">
</div>

In [ ]:
#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(radius, theta, phi, useLonLat=True):
    if useLonLat == True:
        theta, phi = np.radians(theta+180.), np.radians(90. - phi)
    X = radius * np.cos(theta) * np.sin(phi)
    Y = radius * np.sin(theta) * np.sin(phi)
    Z = radius * np.cos(phi)
    
    #Return data either as a list of XYZ coordinates or as a single XYZ coordinate
    if (type(X) == np.ndarray):
        return np.stack((X, Y, Z), axis=1)
    else:
        return np.array([X, Y, Z])

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(XYZ, useLonLat=True):
    X, Y, Z = XYZ[:, 0], XYZ[:, 1], XYZ[:, 2]
    R = (X**2 + Y**2 + Z**2)**0.5
    theta = np.arctan2(Y, X)
    phi = np.arccos(Z / R)
    
    #Return results either in spherical polar or leave it in radians
    if useLonLat == True:
        theta, phi = np.degrees(theta), np.degrees(phi)
        lon, lat = theta - 180, 90 - phi
        lon[lon < -180] = lon[lon < -180] + 360
        return R, lon, lat
    else:
        return R, theta, phi

Now that we have set up our coordinate transformation functions, we use those to plot the earth data on a sphere. To make the landscape more visible, we exegerate the heights topology.

Also, to avoid passing too many variable later on in this project, we store all user specifed properties into a single dictionary. Later we will create a function that only takes *props* as an argument, and the rest of the simulation is taken care of.

In [ ]:
#Exagerate the heights
lon, lat, height = initData[:, 0], initData[:, 1], initData[:, 2]
exageratedRadius = height * props['heightAmplificationFactor'] + props['earthRadius']

#Create XYZ coordinates from polar coordinates
initialEarthXYZ = polarToCartesian(exageratedRadius, lon, lat)

#Create mesh with same faces as the flat initialDataMesh
earthFaces = initialDataMesh.faces
initialEarthMesh = pv.PolyData(initialEarthXYZ, earthFaces)

#Create a plotter for visualizing the round earth
sphereEarthPlotter = pv.Plotter(notebook=True)
sphereEarthPlotter.add_mesh(initialEarthMesh, scalars=height)
sphereEarthPlotter.show(jupyter_backend='panel', window_size=[800, 400])

# Moving Tectonic Plates

To specify which tectonic plate a particular vertex on our sphere belongs to, we create a list of *Plate Ids* where each vertex is given a number based on which plate they belong to. To move tectonic plates, we create a rotation quaternion for each plate, and apply rotations to all vertices based on their plate ids.

We use the library *pygplates* to assign plate ids to vertices. *Pygplates* requires a list of point features for each vertex on our sphere and assigns plate ids to those.

In [ ]:
#Creates a list of point features for each vertex on our sphere
def createPointFeatures(lon, lat):
    pointsOnSphere = [pygplates.PointOnSphere(float(lat[i]), float(lon[i])) for i in range(len(lon))]
    pointFeatures = []
    for point in pointsOnSphere:
        pointFeature = pygplates.Feature()
        pointFeature.set_geometry(point)
        pointFeatures.append(pointFeature)
    return pointFeatures

#Returns a list of plate Ids for points on our sphere
def getPlateIdsAtTime(props, time, pointFeatures):
    assignedPointFeatures = pygplates.partition_into_plates(
        props['platePolygonsDirectory'],
        props['rotationsDirectory'],
        pointFeatures,
        reconstruction_time=float(time),
        properties_to_copy = [
            pygplates.PartitionProperty.reconstruction_plate_id,
            pygplates.PartitionProperty.valid_time_period])
    featureIds = [feat.get_reconstruction_plate_id() for feat in assignedPointFeatures]
    return np.array(featureIds)

#Run the new functions to get plate ids
pointFeatures = createPointFeatures(lon, lat)
plateIds = getPlateIdsAtTime(props, props['startTime'], pointFeatures)

#Create a plot of the earth with vertices color coded by plate ids
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(initialEarthMesh, scalars=plateIds)
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

To move the tectonic plates, we use *pygplates* to get an axis $\textbf{u}$ and angle $\theta$ of rotation. We use those to create a rotation quaternion, and apply the quaternion to all vertices that belong on a particular tectonic plate.

A rotation quaternion is given by:

$\cos \frac{\theta}{2} + (u_x \hat{\textbf{i}} + u_y \hat{\textbf{j}} + u_z \hat{\textbf{k}}) \sin \frac{\theta}{2}$ 

where $\hat{\textbf{i}}, \hat{\textbf{j}}$ and  $\hat{\textbf{k}}$ are unit vectors representing cartesian axis. To apply the rotations, we use the *scipy.spatial.Rotation* library.

In [ ]:
#Returns a rotation quaternion
def quaternion(axis, angle):
    return [np.sin(angle/2) * axis[0], 
            np.sin(angle/2) * axis[1], 
            np.sin(angle/2) * axis[2], 
            np.cos(angle/2)]

#Get stage rotation data from pygplates and return a scipy rotation
def getRotation(time, deltaTime, plateId):
    stageRotation = rotationModel.get_rotation(int(time-deltaTime), int(plateId), int(time))
    stageRotation = stageRotation.get_euler_pole_and_angle()

    #Create rotation quaternion from axis and angle
    axisLatLon = stageRotation[0].to_lat_lon()
    axis = polarToCartesian(1, axisLatLon[1], axisLatLon[0])
    angle = stageRotation[1]
    return R.from_quat(quaternion(axis, angle))

#Move tectonic plates along the sphere by applying rotations to vertices with appropriate plate ids
def movePlates(rotationModel, sphereXYZ, plateIds, time, deltaTime):
    newXYZ = np.copy(sphereXYZ)
    for idx in np.unique(plateIds):
        rot = getRotation(time, deltaTime, idx)
        newXYZ[plateIds == idx] = rot.apply(newXYZ[plateIds == idx])
    return newXYZ

#Create mesh with moved tectonic plates
rotationModel = pygplates.RotationModel(props['rotationsDirectory'])
earthWithMovedPlatesXYZ = movePlates(rotationModel, initialEarthXYZ, plateIds, props['startTime'], props['deltaTime'])
earthWithMovedPlatesMesh = pv.PolyData(earthWithMovedPlatesXYZ, earthFaces)

#Create an earth plot with the meshe's edges displayed
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(earthWithMovedPlatesMesh, scalars=plateIds, show_edges=True)
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

# Remeshing the Sphere

By moving tectonic plates, vertices around plate boundaries may overlap when plates converge or they may move too far from each other when plates diverge. To fix this we will interpolated the heights of the new sphere back onto the polar coordinates of the old sphere.

In [ ]:
#Create template coordinates to interpolate with
templateLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Create reference coordinates of the sphere with moved plates to interpolate from
movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
heights = initData[:, 2]

#Remesh the sphere using interpolation
interpolatedHeights = griddata(movedLonLat, heights, templateLonLat)
whereNAN = np.argwhere(np.isnan(interpolatedHeights))
interpolatedHeights[whereNAN] = griddata(movedLonLat, heights, templateLonLat[whereNAN], method='nearest')

#Create new moved sphere with a better mesh
interpolatedRadius = interpolatedHeights * props['heightAmplificationFactor'] + props['earthRadius']
interpolatedSphereXYZ = polarToCartesian(interpolatedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
interpolatedSphereXYZ = pv.PolyData(interpolatedSphereXYZ, earthFaces)

#Create plot of the remesh
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(interpolatedSphereXYZ, scalars=interpolatedHeights)
plateIdsPlotter.camera_position = 'xz'
plateIdsPlotter.camera.azimuth = 200
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

Hmm... The mountain ranges of South America on the above earth dissappeared!!! What happened? The problem is that the interpolation scheme sampled heights of subducting vertices, but we don't want that. We only want to sample points of the overiding plates.

To fix this issue, we can set heights of subducting vertices to the heights of nearby over-riding vertices. But to do this, we first have to identify vertices that are in the subduction region. To identify these, we transform our coordinates into a cylinder so that vertices **not** on plate boundaries are equally spaced apart. Vertices in the subduction regions will be closer to each other and can therefore be identified by the DBSCAN clustering algorithm. Then, for each vertex belonging to a cluster, we set its height to the maximum of its nearest neighbours.

With $\theta$ and $z$ representing lon/lat coordinates (a flat earth), we use the following coordinate transformation to create a cylinder:

$$
\begin{align}
x & = r \cos \theta \\
y & = r \sin \theta \\
z & = z
\end{align}
$$

We want to chose a radius $r$ such that the vertices form a grid with equal spacing along both lon/lat directions. Let $m$ be the resolution of vertices along the longitudinal direction, and $n$ be the resolution along the latitudinal direction. The circumference of the cylinder is $2 \pi r$, so the spacing along the longitudinal direction will be $\frac{2 \pi r}{m}$. Let $h$ be the distance from the north to south pole, the spacing along the latitudinal direction is $\frac{h}{n}$. We want both spacings to be equal, so our desired cylinder radius is:

$$
\begin{align}
\large \frac{2 \pi r}{m} & = \large \frac{h}{n} \\
\large r & = \large \frac{m h}{2 \pi n}
\end{align}
$$

Within this new cylindrical coordinate system, we can now use the DBSCAN clustering algorithm to identify overlapping plate vertices. Given a threshold distance, the DBSCAN algorithm groups vertices together which are closer than specified distance. Here we chose a threshold distance that is a bit shorter than the normal grid spacing of the vertices:

We provide an example bellow:

In [ ]:
#Coordinate transformation functions from cartesian to cylindrical polar coordinates
def cartesianToCylindrical(X, Y, Z):
    r = (X**2 + Y**2)**0.5
    theta = np.arctan2(Y, X)
    return np.stack((r, theta, Z), axis=1)

#Coordinate transformation functions from cylindrical polar coordinates to cartesian
def cylindricalToCartesian(r, theta, Z, useDegrees=True):
    if useDegrees == True:
        theta = np.radians(theta+180.)
    X = r * np.cos(theta)
    Y = r * np.sin(theta)
    return np.stack((X, Y, Z), axis=1)

props['MaxClusterSize'] = 3

#Get number of subdivisions along lon/lat axii
m = len(np.unique(initData[:, 0])) - 1
n = len(np.unique(initData[:, 1])) - 1
h = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])

#Create clyinder
cylinderRadius = m * h / (np.pi * n * 2)
cylinderXYZ = cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
cylinderMesh = pv.PolyData(cylinderXYZ)

#Run the clustering algorithm
spacing = 300 / m
cluster = DBSCAN(eps=spacing, min_samples=props['MaxClusterSize'], n_jobs=4).fit(cylinderXYZ)
isCluster = (cluster.labels_ != -1)

#Points near the poles are clustered, even when they are not plate boundaries, so we manually set those to False
isCluster[np.abs(initData[:, 1])>80] = False

#Show results
pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.Cylinder(radius=cylinderRadius-0.5, height=180, direction=(0, 0, 1)))
pl.add_mesh(cylinderMesh, scalars=isCluster)
pl.show(jupyter_backend='panel', window_size=[800, 400])

Now that we have identified which vertices bellong to converging plate boundaries, we set those vertices to the maximum of their nearest neighbours. We do this in the code bellow, and visualize results by showing the new heights in yellow, and old heights in red. Feel free to pan the viewport camera to see results more clearly.

In [ ]:
numOfNeighbsToConsider = 6

#Create KDTree to find nearest neighbours of each point in cluster
pointsInClusterLonLat = cylinderXYZ[isCluster]
clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=numOfNeighbsToConsider+1)

#Get heights of nearest neighbours
heightsInCluster = heights[isCluster]
clusterPointsNeighboursId = clusterKDTree[1]
neighbourHeights = heightsInCluster[clusterPointsNeighboursId[:, 1:]]

#For points in cluster, set new heights to the maximum height of nearest neighbours
newHeights = np.copy(heights)
newHeights[isCluster] = np.max(neighbourHeights, axis=1)

#Create meshes for plot
flatEarthXYZ = np.stack((movedLonLat[:, 0], movedLonLat[:, 1], heights), axis=1)
newFlatEarthXYZ = np.stack((movedLonLat[:, 0], movedLonLat[:, 1], newHeights), axis=1)

#Show results
pl = pv.Plotter(notebook=True)
pl.add_mesh(flatEarthXYZ[isCluster], color='red')
pl.add_mesh(newFlatEarthXYZ, scalars=isCluster)
pl.camera_position = 'xy'
pl.camera.position = (0, 0, 350.0)
pl.show(jupyter_backend='panel', window_size=[800, 400])

Now that we have demonstrated how the remesh algorithm works, we create functions of the algorithm. We provide an example of the final remesh, but this time with **no** disappearing mountain ranges.

In [ ]:
props['thetaResolution'] = len(np.unique(initData[:, 0])) - 1
props['phiResolution'] = len(np.unique(initData[:, 1])) - 1
props['numOfNeighbsToConsider'] = 6
props['clusterThresholdProportion'] = 300 / 360

def getHeightsForRemesh(props, movedLonLat, heights):
    m = props['thetaResolution']
    n = props['phiResolution']
    numOfNeighbs = props['numOfNeighbsToConsider']
    h = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    
    #Create clyinder
    cylinderRadius = m * h / (np.pi * n * 2)
    cylinderXYZ = cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])

    #Run the clustering algorithm
    threshHoldDist = props['clusterThresholdProportion'] * 360 / m
    cluster = DBSCAN(eps=threshHoldDist, min_samples=3).fit(cylinderXYZ)
    isCluster = (cluster.labels_ != -1)
    
    #Create KDTree to find nearest neighbours of each point in cluster
    pointsInClusterLonLat = cylinderXYZ[isCluster]
    clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=numOfNeighbs+1)
    
    #Get heights of nearest neighbours
    heightsInCluster = heights[isCluster]
    clusterPointsNeighboursId = clusterKDTree[1]
    neighbourHeights = heightsInCluster[clusterPointsNeighboursId[:, 1:]]

    #For points in cluster, set new heights to the maximum height of nearest neighbours
    newHeights = np.copy(heights)
    newHeights[isCluster] = np.max(neighbourHeights, axis=1)
    return newHeights

def remeshSphere(props, templateLonLat, movedLonLat, heights):
    heightsForRemesh = getHeightsForRemesh(props, movedLonLat, heights)
    newHeights = griddata(movedLonLat, heightsForRemesh, templateLonLat)
    whereNAN = np.argwhere(np.isnan(newHeights))
    newHeights[whereNAN] = griddata(movedLonLat, heightsForRemesh, templateLonLat[whereNAN], method='nearest')
    return newHeights

#Create template coordinates to interpolate with
templateLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Create reference coordinates of the sphere with moved plates to interpolate from
movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
heights = initData[:, 2]

#Remesh the sphere using interpolation
interpolatedHeights = remeshSphere(props, templateLonLat, movedLonLat, heights)

#Create new moved sphere with the better mesh
interpolatedRadius = interpolatedHeights * props['heightAmplificationFactor'] + props['earthRadius']
interpolatedSphereXYZ = polarToCartesian(interpolatedRadius, templateLonLat[:, 0], templateLonLat[:, 1])
interpolatedSphereXYZ = pv.PolyData(interpolatedSphereXYZ, earthFaces)

#Show results
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(interpolatedSphereXYZ, scalars=interpolatedHeights)
plateIdsPlotter.camera_position = 'xz'
plateIdsPlotter.camera.azimuth = 200
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
def remeshSphere(templateLonLat, movedLonLat, heights):
    newHeights = griddata(movedLonLat, heights, templateLonLat)
    whereNAN = np.argwhere(np.isnan(newHeights))
    newHeights[whereNAN] = griddata(movedLonLat, heights, templateLonLat[whereNAN], method='nearest')
    return newHeights

#Create template coordinates to interpolate with
templateLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Create reference coordinates of the sphere with moved plates to interpolate from
movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
heights = initData[:, 2]

#Remesh the sphere using interpolation
interpolatedHeights = remeshSphere(templateLonLat, movedLonLat, heights)

#Create new moved sphere with a better mesh
interpolatedRadius = interpolatedHeights * props['heightAmplificationFactor'] + props['earthRadius']
interpolatedSphereXYZ = polarToCartesian(interpolatedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
interpolatedSphereXYZ = pv.PolyData(interpolatedSphereXYZ, earthFaces)

#Get spherical coordinates of moved plates
R, theta, phi = cartesianToPolarCoords(earthWithMovedPlatesXYZ, useLonLat=True)
h = np.max(phi) - np.min(phi)
radi = m * h / (np.pi * n * 2)

cylinderRadius = radi
cylinderXYZ = cylindricalToCartesian(cylinderRadius,  theta, phi)
cylinderMesh = pv.PolyData(cylinderXYZ)

pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.Cylinder(radius=cylinderRadius-0.5, height=180, direction=(0, 0, 1)))
pl.add_mesh(cylinderMesh, color='blue')
pl.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
numOfVerts = movedLonLat.shape[0]

print(180*3)
print(((numOfVerts-541)/2)**0.5)
lon, lat, heights = initData[:, 0], initData[:, 1], initData[:, 2]
print(len(np.unique(lon)))
print(len(np.unique(lat)))
m = len(np.unique(lon))
n = len(np.unique(lat))
print(2 * (n-1) / (m-1))

def cartesianToCylindrical(X, Y, Z):
    r = (X**2 + Y**2)**0.5
    theta = np.arctan2(Y, X)
    return np.stack((r, theta, Z), axis=1)

def cylindricalToCartesian(r, theta, Z, useDegrees=True):
    if useDegrees == True:
        theta = np.radians(theta+180.)
    X = r * np.cos(theta)
    Y = r * np.sin(theta)
    return np.stack((X, Y, Z), axis=1)

m = len(np.unique(initData[:, 0])) - 1
n = len(np.unique(initData[:, 1])) - 1
print(m)
print(n)
radi = ((m * n)**0.5) / np.pi
print(radi)

cylinderRadius = 40
lon, lat, one = movedLonLat[:, 0], movedLonLat[:, 1], np.ones(len(movedLonLat[:, 0]))
cylinderRad = one * cylinderRadius

cylinderXYZ = cylindricalToCartesian(cylinderRadius, lon, lat)
cylinderMesh = pv.PolyData(cylinderXYZ)

unitPlotter = pv.Plotter(notebook=True)
#unitPlotter.add_mesh(pv.Sphere(radius=0.95))
unitPlotter.add_mesh(pv.Cylinder(radius=cylinderRadius-0.5, height=180, direction=(0, 0, 1)))
unitPlotter.add_mesh(cylinderXYZ, color='b')#, scalars=interpolatedHeights)
unitPlotter.add_axes(interactive=True)
unitPlotter.show(jupyter_backend='panel', window_size=[800, 400])


m, n, r = 600, 180, 1
sphreMesh = pv.Sphere(radius=r, theta_resolution=m, phi_resolution=n)
sphreXYZ = sphreMesh.points

R, theta, phi = cartesianToPolarCoords(sphreXYZ, useLonLat=True)
h = np.max(phi) - np.min(phi)
radi = m * h / (np.pi * n * 2)

cylinderRadius = radi
cylinderXYZ = cylindricalToCartesian(cylinderRadius,  theta, phi)
cylinderMesh = pv.PolyData(cylinderXYZ)



pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.Cylinder(radius=cylinderRadius-0.5, height=180, direction=(0, 0, 1)))
pl.add_mesh(cylinderXYZ, color='blue')
pl.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
unitSphereXYZ = polarToCartesian(1, movedLonLat[:, 0], movedLonLat[:, 1])
unitSphereMesh = pv.PolyData(unitSphereXYZ)

unitPlotter = pv.Plotter(notebook=True)
unitPlotter.add_mesh(pv.Sphere(radius=0.95))
unitPlotter.add_mesh(unitSphereMesh, color='b')#, scalars=interpolatedHeights)
unitPlotter.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
class Earth:
    def __init__(self, props):
        self.props = props
        self.earthRadius = 6371
        
        #Set initial earth data
        initData = getInitialEarth(props['startTime'])
        self.lonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)
        self.sphereXYZ = polarToCartesian(1, initData[:, 0], initData[:, 1])
        self.heightHistory = [initData[:, 2]]
        
        self.simulationTimes = np.arange(props['startTime'], props['endTime']-props['deltaTime'], -props['deltaTime'])
        self.rotationModel = pygplates.RotationModel(props['rotationsDirectory'])
        self.pointFeatures = createPointFeatures(initData[:, 0], initData[:, 1])
        self.earthFaces = pv.PolyData(initData).delaunay_2d().faces
    
    def getEarthMesh(self):
        amplifier = props['heightAmplificationFactor']
        lon, lat = self.lonLat[:, 0], self.lonLat[:, 1]
        exageratedRadius = self.heightHistory[-1] * amplifier + self.earthRadius
        earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
        earthMesh = pv.PolyData(earthXYZ, self.earthFaces)
        return earthMesh
    
    def showEarth(self):
        earth = self.getEarthMesh()
        plotter = pv.PlotterITK()
        plotter.add_mesh(earth, scalars=self.heightHistory[-1])
        plotter.show(window_size=[800, 400])
    
    def runTectonicSimulation(self):
        for time in self.simulationTimes:
            plateIds = getPlateIdsAtTime(self.props, time, self.pointFeatures)
            movedEarthXYZ = movePlates(self.rotationModel, self.sphereXYZ, plateIds, time, self.props['deltaTime'])
            movedLonLat = cartesianToPolarCoords(movedEarthXYZ)
            movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
            
            heights = remeshSphere(self.lonLat, movedLonLat, self.heightHistory[-1])
            self.heightHistory.append(heights)
        

earth = Earth(props)
earth.showEarth()

In [ ]:
earth.runTectonicSimulation()
earth.showEarth()

In [ ]:
#Properties representing time range and time steps of our simulation
props['startTime'] = 10
props['endTime'] = 0
props['deltaTime'] = 5        

#Get initial data
initData = getInitialEarth(props['startTime'])
sphereXYZ = polarToCartesian(1, initData[:, 0], initData[:, 1])
initLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Append initial earth to earth history
lon, lat, heights = initData[:, 0], initData[:, 1], initData[:, 2]
heightHistory.append(heights)

pointFeatures = createPointFeatures(lon, lat)
simulationTimes = np.arange(props['startTime'], props['endTime']-props['deltaTime'], -props['deltaTime'])

for time in simulationTimes:
    #Get plate ids
    plateIds = getPlateIdsAtTime(props, time, pointFeatures)

    #Create mesh with moved tectonic plates
    rotationModel = pygplates.RotationModel(props['rotationsDirectory'])
    earthWithMovedPlatesXYZ = movePlates(rotationModel, sphereXYZ, plateIds, time, props['deltaTime'])
    movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
    movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
    
    heights = remeshSphere(initLonLat, movedLonLat, heights)
    heightHistory.append(heights)
    print(time)
    
heightHistory = np.array(heightHistory)

print(initialEarthXYZ.shape)
print(earthWithMovedPlatesXYZ.shape)

In [ ]:
i = 2

exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
sphereMesh = pv.PolyData(earthXYZ, earthFaces)

plateIdsPlotter = pv.PlotterITK()
plateIdsPlotter.add_mesh(sphereMesh, scalars=heightHistory[i])
plateIdsPlotter.show()

#Create plot
#plateIdsPlotter = pv.Plotter(notebook=True)
#plateIdsPlotter.add_mesh(sphereMesh, scalars=heightHistory[i])
#plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
#Create an earth plot with the meshe's edges displayed
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(interpolatedSphereXYZ, scalars=interpolatedHeights)
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

In [ ]:
i = 2

exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
sphereMesh = pv.PolyData(earthXYZ, earthFaces)

plateIdsPlotter.update_coordinates(earthXYZ)

In [ ]:
exageratedRadius = heightHistory[0] * props['heightAmplificationFactor'] + props['earthRadius']
earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
plateIdsPlotter.update_coordinates(earthXYZ)

In [ ]:
import itk
from itkwidgets import view

exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
sphereMesh = pv.PolyData(earthXYZ, earthFaces)
sphereMesh['scallars'] = heightHistory[i]
plot = view(geometries=sphereMesh)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plotResults(i):
    exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
    earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
    sphereMesh = pv.PolyData(earthXYZ, earthFaces)
    
    sphereMesh['scallars'] = heightHistory[i]
    return view(geometries=sphereMesh)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plotResults(i):
    exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
    earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
    sphereMesh = pv.PolyData(earthXYZ, earthFaces)
    
    sphereMesh['scallars'] = heightHistory[i]
    return view(geometries=sphereMesh)

    #plateIdsPlotter = pv.PlotterITK()
    #plateIdsPlotter.add_mesh(sphereMesh, scalars=heightHistory[i])
    #return plateIdsPlotter.show()

slider = widgets.IntSlider(min=np.min(simulationTimes), max=np.max(simulationTimes), step=props['deltaTime'], value=0)
interact(plotResults, i=slider)

In [ ]:
import ipywidgets as widgets
from jp_doodle import dual_canvas
from IPython.display import display

# create a triangle diagram
swatch = dual_canvas.swatch(pixels=200)
points = [[-1,-1], [0,1], [1, -1]]
triangle = swatch.polygon(points, color="red", name="triangle")
swatch.fit(margin=10)

In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

#JupyterDash.infer_jupyter_proxy_config()

app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id='plot')
    ])
])

app.run_server(mode="inline")

In [ ]:
from urllib.request import urlretrieve
import os

import itk

from itkwidgets import view

# Download data
file_name = '005_32months_T2_RegT1_Reg2Atlas_ManualBrainMask_Stripped.nrrd'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/564a5b078d777f7522dbfaa6/download'
    urlretrieve(url, file_name)

image = itk.imread(file_name)
print(type(image))
view(image)#, axes=True, vmin=4000, vmax=17000, gradient_opacity=0.9)


In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )

# Run app and display result inline in the notebook
app.run_server(mode='inline', port=1002, host='localhost:1000/')

In [ ]:


def getMeshAtTime(time):
    kdTree = KDTree(simulationTimes.reshape(-1, 1))
    j = kdTree.query([time])[1]
    t = simulationTimes[j]
    i = np.argwhere(simulationTimes==t)
    exageratedRadius = heightHistory[i] * props['heightAmplificationFactor'] + props['earthRadius']
    earthXYZ = polarToCartesian(exageratedRadius[0][0], lon, lat)
    sphereMesh = pv.PolyData(earthXYZ, earthFaces)
    scallars = heightHistory[i]
    return sphereMesh, scallars, earthXYZ


#simulationTimes
#Class for an interactive pyvista plot
class earthRenderer():
    def __init__(self, time):
        self.kwargs = {'time': 0}
        self.plotter = pv.PlotterITK()
        
        
        self.mesh, self.scallars, XYZ = getMeshAtTime(time)
        
        #Add main mesh
        #self.mesh, self.scallars, XYZ = getMeshAtTime(time)
        self.plotter.add_mesh(self.mesh)#, scalars=self.scallars)#, cmap='gist_earth')
        
        #Add boundary mesh
        #boundaryData = earthInit.getPlateBoundaryData(time, platePolygonsDirectory, rotationsDirectory, earthRadius)
        #self.boundActor = self.plotter.add_mesh(boundaryData[0], color='b')
        
        #Add slider to specify time with
        self.plotter.add_slider_widget(
            callback = lambda value: self('time', value),
            rng = simulationTimes,
            value = 5,
            title = 'Time',
            style = 'modern'
        )
        
    #Function to be called by slider updates
    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update()
    
    #Update when slider changes values
    def update(self):
        time = self.kwargs['time']
        sphereMesh, scallars, XYZ = getMeshAtTime(time)
        self.plotter.update_coordinates(XYZ, mesh=self.mesh)
        self.plotter.update_scalars(scallars, mesh=self.mesh)
        self.plotter.remove_actor(self.boundActor)
        #boundaryData = earthInit.getPlateBoundaryData(time, platePolygonsDirectory, rotationsDirectory, earthRadius)
        #self.boundActor = self.plotter.add_mesh(boundaryData[0], color='b')
        return

#Initiate an instance of the earth renderer
earth = earthRenderer(simulationTimes[-1])
earth.plotter.show()

In [ ]:
p = pv.PlotterITK()
print(p.__dict__)
print()
print(pv.PlotterITK.__dict__)

In [ ]:
'''
#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(*coords, radians=False, radius=1):
    if radians == False:
        coords = np.array(coords)
        Theta, Phi = np.radians(coords[:, 0]+180), np.radians(90 - coords[:, 1])
    X = radius * np.cos(Theta) * np.sin(Phi)
    Y = radius * np.sin(Theta) * np.sin(Phi)
    Z = radius * np.cos(Phi)
    return np.stack((X, Y, Z), axis=1)

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(X, Y, Z):
    R = (X**2 + Y**2 + Z**2)**0.5
    Theta = np.arctan2(Y, X)
    Phi = np.arccos(Z / R)
    return R, Theta, Phi

#Takes longatude and latitude coordinates and converts them to cartesian coordinates
def lonLatToCartesian(lon, lat, radius=1.0):
    X, Y, Z = polarToCartesian(radius, np.radians(lon+180), np.radians(90 - lat))
    return np.stack((X, Y, Z), axis=-1)

#Takes cartesian coordinates and converts them to longatude and latitude
def cartesianToLonLat(X, Y, Z):
    r, theta, phi = cartesianToPolarCoords(X, Y, Z)
    theta, phi = np.degrees(theta), np.degrees(phi)
    lon, lat = theta - 180, 90 - phi
    lon[lon < -180] = lon[lon < -180] + 360
    return lon, lat

#Function for moving vertices on a sphere along the radial direction by amount of delta radius (dr)
def moveAlongRadialDirection(XYZ, dr, inputType='cartesian'):
    if inputType=='lonLat' or XYZ.shape[1]==2:
        XYZ = lonLatToCartesian(XYZ[:, 0], XYZ[:, 1], radius=earthRadius)
    r, theta, phi = cartesianToPolarCoords(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2])
    newSphereX = XYZ[:, 0] + np.cos(theta) * np.sin(phi) * dr
    newSphereY = XYZ[:, 1] + np.sin(theta) * np.sin(phi) * dr
    newSphereZ = XYZ[:, 2] + np.cos(phi) * dr
    return np.stack((newSphereX, newSphereY, newSphereZ), axis=-1)

#Given a sphere XYZ coordinates, we set the radius of all coordinates
def setRadialComponent(XYZ, r):
    if XYZ.shape[1]==2:
        XYZ = polarToCartesian(XYZ)
    rad, theta, phi = cartesianToPolarCoords(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2])
    newSphereX = np.cos(theta) * np.sin(phi) * r
    newSphereY = np.sin(theta) * np.sin(phi) * r
    newSphereZ = np.cos(phi) * r
    return np.stack((newSphereX, newSphereY, newSphereZ), axis=-1)
'''

In [ ]:
'''
#Read initial landscape data from file which is in the form of (lon, lat, height)
def getInitialEarth(time, paleoDemsPath='./PaleoDEMS'):
    paleoDemsPath = Path('./PaleoDEMS')
    initialLandscapePath = list(paleoDemsPath.glob('**/*%03.fMa.csv'%startTime))[0]
    initialLandscapeFileLines = open(initialLandscapePath).read().split('\n')[1:-1]
    initLandscapeData = [line.split(',') for line in initialLandscapeFileLines]
    initLandscapeData = np.array(initLandscapeData).astype(float)
    initLandscapeData[:, 2] /= 1000
    heightMap = initLandscapeData[:, 2]
    return initLandscapeData, heightMap

def createEarthMesh(coords, coordType='cartesian', earthRadius=6371):
    if coordType == 'LonLatHeight':
        lon, lat, heights = coords[:, 0], coords[:, 1], coords[:, 2]
        heights = coords[:, 2]
        coords = coords[:, :2]
    sphereWithHeights = setRadialComponent(coords, heights * 40 + earthRadius)
    lonLat3D = np.stack((lon, lat, np.zeros(len(lon))), axis=1)
    sphereFaces = pv.PolyData(lonLat3D).delaunay_2d().faces
    sphereMesh = pv.PolyData(sphereWithHeights, sphereFaces)
    return sphereMesh
    
startTime = 100 #MYA
earthRadius = 6371

initLandscapeLLH, heightMap = getInitialEarth(startTime)
sphereMesh = createEarthMesh(initLandscapeLLH, coordType='LonLatHeight')

plotter = pv.PlotterITK()
plotter.add_mesh(sphereMesh, scalars=heightMap)#, cmap='gist_earth')
plotter.show()
'''